In [33]:
from bs4 import BeautifulSoup as bs
import requests as rq

In [45]:
def extract_thread_url(thread): return thread.findAll('a')[0]['href']

In [42]:
def fetch_thread_html(thread_url):
    try:
        return rq.get(thread_url).text
    except requests.exceptions.RequestException as e:
        print (e)

In [49]:
def get_bs_object(html): return bs(html, 'lxml')

In [48]:
def open_html_file(file_name): return open(file_name)

In [53]:
def get_page_title_html(soup_html): return soup_html.findAll('td', { 'class' : 'f_title' })

In [74]:
def get_fieldset_rows(soup_html): return soup_html.findAll('tr')

In [83]:
def get_row_cols(row): return [col for col in row.findAll('td')]

In [84]:
html_page = get_bs_object(open_html_file('eforum/forumdisplay.php_fid=13_page=1.html'))
thread_titles = get_page_title_html(html_page)
thread_urls = extract_thread_urls(thread_titles)
thread_bs_html = get_bs_object(fetch_thread_html(thread_urls[4]))
fieldset = thread_bs_html.find('fieldset')
fieldset_rows = get_fieldset_rows(fieldset)
cols = get_row_cols(fieldset_rows[0])
cols[-3:]

[<td>  Acceptance</td>, <td>  <b>+1</b></td>, <td>  Woaw!</td>]